In [1]:
import sys
import os
sys.path.append(os.path.abspath('../../stratipy'))
from stratipy import load_data
import importlib  # NOTE for python >= Python3.4
import scipy.sparse as sp
import numpy as np
import pandas as pd
import time
from datetime import datetime
# from sklearn.model_selection import ParameterGrid
from scipy.io import loadmat, savemat
import networkx as nx
import collections
import matplotlib.pyplot as plt
from matplotlib_venn import venn2
%pylab inline
%matplotlib inline
%load_ext autoreload
%autoreload 2

Populating the interactive namespace from numpy and matplotlib


/Users/Min/anaconda/envs/stratipy3/lib/python3.6/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['datetime']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [2]:
pd.options.mode.chained_assignment = None  # default='warn'
data_folder = "dataset/"

# Load APID PPI data (Dr. Javier DE LAS RIVAS)
Ref: __[APID interactomes: providing proteome-based interactomes with controlled quality for multiple species and derived networks](https://academic.oup.com/nar/article/44/W1/W529/2499348)__<br>

APID server: http://apid.dep.usal.es/<br>
Last downloaded on 2018/04(Apr)/05 <br>
Level 2: proven by 2 experiments or more:
- 133 993 interactions
- 17 324 interacting proteins


#### Raw colums:
- InteractionID	
- UniprotID_A	
- UniprotName_A	
- GeneName_A	
- UniprotID_B	
- UniprotName_B	
- GeneName_B	
- Experiments: number of experiments
- Methods: number of methods
- Publications: number of publications
- 3DStructures: number of 3D structures (PDBs)
- CurationEvents: number of curation events (including source databases)



In [3]:
df_apid = pd.read_csv(data_folder + "9606_Q2.txt", sep="\t")
print("interacting proteins number:", df_apid.shape[0])

# keep only interactions between Human-Human proteins
df_apid_HH = df_apid[df_apid.UniprotName_A.str.contains("_HUMAN") & 
                     df_apid.UniprotName_B.str.contains("_HUMAN") == True]
print("interacting Human-Human proteins number:", df_apid_HH.shape[0])

print("{} interspecific interactions removed ({}%)"
      .format(df_apid.shape[0]-df_apid_HH.shape[0],
             round((1-df_apid_HH.shape[0]/df_apid.shape[0])*100,1)))

df_apid_HH.head()

interacting proteins number: 133993
interacting Human-Human proteins number: 115981
18012 interspecific interactions removed (13.4%)


,InteractionID,UniprotID_A,UniprotName_A,GeneName_A,UniprotID_B,UniprotName_B,GeneName_B,Experiments,Methods,Publications,3DStructures,CurationEvents
0,242748,A0A024R0Y4,A0A024R0Y4_HUMAN,TADA2A,Q8WWY3,PRP31_HUMAN,PRPF31,3,3,1,0,3
1,277739,A0A024R4Q5,A0A024R4Q5_HUMAN,TFPT,Q96D03,DDT4L_HUMAN,DDIT4L,2,2,1,0,4
2,277740,A0A024R4Q5,A0A024R4Q5_HUMAN,TFPT,P51911,CNN1_HUMAN,CNN1,2,2,1,0,4
3,302485,A0A024R5S0,A0A024R5S0_HUMAN,hCG_2003792,Q9NVV9,THAP1_HUMAN,THAP1,3,3,1,0,3
4,321925,A0A024R5S0,A0A024R5S0_HUMAN,hCG_2003792,Q02446,SP4_HUMAN,SP4,3,3,1,0,3


In [4]:
# keep only GeneSymbol ID and UniProt ID
# then change column names
df_apid_HH = df_apid_HH[["GeneName_A", "UniprotID_A", "GeneName_B", "UniprotID_B"]]
df_apid_HH.columns = ["GeneSymbol_1", "UniProt_1", "GeneSymbol_2", "UniProt_2"]
df_apid_HH.head()

,GeneSymbol_1,UniProt_1,GeneSymbol_2,UniProt_2
0,TADA2A,A0A024R0Y4,PRPF31,Q8WWY3
1,TFPT,A0A024R4Q5,DDIT4L,Q96D03
2,TFPT,A0A024R4Q5,CNN1,P51911
3,hCG_2003792,A0A024R5S0,THAP1,Q9NVV9
4,hCG_2003792,A0A024R5S0,SP4,Q02446


### There are some Gene Symbols not found

In [5]:
# APID's UniProt
apid_unip_1 = df_apid_HH.UniProt_1.drop_duplicates()
apid_unip_2 = df_apid_HH.UniProt_2.drop_duplicates()
apid_unip_rows_1 = list(apid_unip_1.index)
apid_unip_rows_2 = list(apid_unip_2.index)
apid_unip = apid_unip_1.append(apid_unip_2).drop_duplicates()

# APID's Gene Symbol
apid_geneSymb = df_apid_HH.GeneSymbol_1.append(df_apid_HH.GeneSymbol_2)
apid_geneSymb = apid_geneSymb.drop_duplicates()

print("unique UniProt number:", len(apid_unip))
print("unique Gene Symbol number:", len(apid_geneSymb))
diff = len(apid_unip)-len(apid_geneSymb)
print("there are {} fewer Gene Symbols than UniProt ID ({}%)"
      .format(diff, round((diff/len(apid_unip))*100, 1)))

unique UniProt number: 14657
unique Gene Symbol number: 14241
there are 416 fewer Gene Symbols than UniProt ID (2.8%)


In [6]:
# get index of NaN values in GeneSymbol_1 & 2
null_columns = df_apid_HH.columns[df_apid_HH.isnull().any()]
print("----- NaN values in Gene Symbols -----")
print(df_apid_HH[null_columns].isnull().sum())

df_apid_HH[df_apid_HH.isnull().any(axis=1)][null_columns]
null_idx = list(df_apid_HH[df_apid_HH.isnull().any(axis=1)][null_columns].index)

df_apid_HH.loc[null_idx].head()

----- NaN values in Gene Symbols -----
GeneSymbol_1    126
GeneSymbol_2    135
dtype: int64


,GeneSymbol_1,UniProt_1,GeneSymbol_2,UniProt_2
205,FSD2,A1L4K1,NaN,Q9H6F0
246,TARSL2,A2RTX5,NaN,B2RDX5
494,NaN,A8K2R3,KRT38,O76015
495,NaN,A8K2R3,TRIM54,Q9BYV2
496,NaN,A8K2R3,TRIM39,Q9HCM9


Sum of NaN values in GeneSymbol_1&2 (258+209=467) < 632 <br>
<b> => One Gene Symbol can have several UniProt ID </b>

## Load UniProtKB data

Complete human gene data set downloaded from __[UniProtKB](ftp://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/idmapping/)__<br>
"HUMAN_9606_idmapping_selected.tab.gz" (2018/01/31)

#### Raw colums:
1. UniProtKB-AC
2. UniProtKB-ID
3. GeneID (EntrezGene)
4. RefSeq
5. GI
6. PDB
7. GO
8. UniRef100
9. UniRef90
10. UniRef50
11. UniParc
12. PIR
13. NCBI-taxon
14. MIM
15. UniGene
16. PubMed
17. EMBL
18. EMBL-CDS
19. Ensembl
20. Ensembl_TRS
21. Ensembl_PRO
22. Additional PubMed

In [7]:
# only get UniProtKB-AC and GeneID (EntrezGene)
df_unip = pd.read_csv(data_folder + "HUMAN_9606_idmapping_selected.tab", sep="\t",
                      index_col=False, usecols=[0,2], names=["UniProt_AC", "EntrezGene"])
df_unip.head()

,UniProt_AC,EntrezGene
0,P31946,7529
1,P62258,7531
2,Q04917,7533
3,P61981,7532
4,P31947,2810


## Load HGNC data
Complete HGNC dataset downloaded from __[HGNC (HUGO Gene Nomenclature Committee)](https://www.genenames.org/cgi-bin/statistics)__ <br>
"hgnc_complete_set.txt" (2018/02(Feb)/09)

#### Raw colums:
1. hgnc_id	
2. symbol	
3. name	
4. locus_group	
5. locus_type	
6. status	
7. location	
8. location_sortable	
9. alias_symbol	
10. alias_name	
11. prev_symbol	
12. prev_name	
13. gene_family	
14. gene_family_id	
15. date_approved_reserved	
16. date_symbol_changed	
17. date_name_changed	
18. date_modified	
19. entrez_id	
20. ensembl_gene_id	
21. vega_id	
22. ucsc_id	
23. ena	
24. refseq_accession	
25. ccds_id	
26. uniprot_ids	
27. pubmed_id	
28. mgd_id	
29. rgd_id	
30. lsdb	
31. cosmic	
32. omim_id	
33. mirbase	
34. homeodb	
35. snornabase	
36. bioparadigms_slc	
37. orphanet	
38. pseudogene.org	
39. horde_id	
40. merops	
41. imgt	
42. iuphar	
43. kznf_gene_catalog	
44. mamit-trnadb	
45. cd	
46. lncrnadb	
47. enzyme_id	
48. intermediate_filament_db	
49. rna_central_ids


In [8]:
# only get symbol (Gene Symbol), entrez_id (EntrezGene) and uniprot_ids
df_hgnc = pd.read_csv(data_folder + "hgnc_complete_set.txt", sep="\t",
                      index_col=False, usecols=[1,18,25])

print("----- NaN values in raw data ({} rows) -----".format(df_hgnc.symbol.size))
df_hgnc.isnull().sum()

----- NaN values in raw data (42471 rows) -----


symbol             0
entrez_id       1267
uniprot_ids    22448
dtype: int64

# ID mapping with APID

### HGNC: from Gene Symbol to EntrezGene ID
#### Overlapping analysis between Gene Symobl and EntrezGene

In [11]:
# Gene Symbols in APID
print("all unique Gene Symbols in APID:", len(apid_geneSymb))

# Gene Symbols in HGNC 
print("all Gene Symbols in HGNC:", df_hgnc.symbol.size)

all unique Gene Symbols in APID: 14241
all Gene Symbols in filtered HGNC: 42471


In [12]:
intersection_geneSymb = list((set(apid_geneSymb)).intersection(set(df_hgnc.symbol)))
print("intersection of Gene Symbols:", len(intersection_geneSymb))

# print("{} of APID's UnitProt ID are not found in EntrezGene ID "
#       .format(len(apid_unip)-len(intersection_prot)))

intersection of Gene Symbols: 14085


In [14]:
# merge on APID's GeneSymbol (1 then 2) to get corresponding EntrezGene ID
# keeping the index from the 1st frame
joined1 = df_apid_HH.reset_index().merge(
    df_hgnc, how='left', left_on='GeneSymbol_1', right_on='symbol').set_index('index')
joined2 = joined1.reset_index().merge(
    df_hgnc, how='left', left_on='GeneSymbol_2', right_on='symbol').set_index('index')

# remove redundant columns
df = joined2.drop(['symbol_x', 'uniprot_ids_x', 'symbol_y', 'uniprot_ids_y'], axis=1)

# rename columns
df = df.rename(columns={'entrez_id_x': 'EntrezGene_1',
                        'entrez_id_y': 'EntrezGene_2'})
# reorder columns
cols = list(df)
cols.insert(0, cols.pop(cols.index('EntrezGene_1')))
cols.insert(3, cols.pop(cols.index('EntrezGene_2')))
df = df.loc[:, cols]

df.head()

,EntrezGene_1,GeneSymbol_1,UniProt_1,EntrezGene_2,GeneSymbol_2,UniProt_2
index,,,,,,
0,6871.0,TADA2A,A0A024R0Y4,26121.0,PRPF31,Q8WWY3
1,29844.0,TFPT,A0A024R4Q5,115265.0,DDIT4L,Q96D03
2,29844.0,TFPT,A0A024R4Q5,1264.0,CNN1,P51911
3,NaN,hCG_2003792,A0A024R5S0,55145.0,THAP1,Q9NVV9
4,NaN,hCG_2003792,A0A024R5S0,6671.0,SP4,Q02446


In [15]:
df.isnull().sum().GeneSymbol_1

126

In [16]:
# Get Gene Symbols with which EntrezGene are NaN
df_geneSymb_entrezNan_1 = df[df.EntrezGene_1.isnull()].GeneSymbol_1
df_geneSymb_entrezNan_2 = df[df.EntrezGene_2.isnull()].GeneSymbol_2

# remove duplicates
df_geneSymb_entrezNan_1 = df_geneSymb_entrezNan_1.drop_duplicates()
df_geneSymb_entrezNan_2 = df_geneSymb_entrezNan_2.drop_duplicates()

df_geneSymb_entrezNan = df_geneSymb_entrezNan_1.append(df_geneSymb_entrezNan_2).drop_duplicates()

print("There are {} ({}%) missing EntrezGene ID via Gene Symbol ID mapping"
     .format(len(df_geneSymb_entrezNan),
             round((len(df_geneSymb_entrezNan)/len(apid_geneSymb))*100, 1)))

There are 159 (1.1%) missing EntrezGene ID via Gene Symbol ID mapping


### UniProtKB-AC: from UniProt ID to EntrezGene ID
#### For Gene Symbols not previously found in APID

In [17]:
# UniProtKB
unip_entrez = df_unip.UniProt_AC.drop_duplicates()
print('all UniProtKB ID:', len(unip_entrez))

# ADIP
apid_unip = df_apid_HH.UniProt_1.append(df_apid_HH.UniProt_2)
apid_unip = apid_unip.drop_duplicates()

overlap_uniprot_adip = (len(apid_unip)/len(unip_entrez))*100
print("{} interacting proteins in ADIP ({}%)"
      .format(len(apid_unip), round(overlap_uniprot_adip, 1)))

all UniProtKB ID: 161566
14657 interacting proteins in ADIP (9.1%)


In [23]:
# data with NaN in GeneSymbol_1 or 2
df_apid_geneSymb_nan1 = df[df.GeneSymbol_1.isnull()]
df_apid_geneSymb_nan2 = df[df.GeneSymbol_2.isnull()]

# merge on APID's UniProt_1 and 2 SEPARATELY to get corresponding EntrezGene ID
# keeping the index from the 1st frame
joined_1 = df_apid_geneSymb_nan1.reset_index().merge(
    df_unip, how='left', left_on='UniProt_1', right_on='UniProt_AC').set_index('index')
joined_2 = df_apid_geneSymb_nan2.reset_index().merge(
    df_unip, how='left', left_on='UniProt_2', right_on='UniProt_AC').set_index('index')

# replace new EntrezGene values found in UniProtKB
df.EntrezGene_1[joined_1.index] = joined_1.EntrezGene
df.EntrezGene_2[joined_2.index] = joined_2.EntrezGene

In [24]:
# rows with both GeneSymbol_1&2 are NaN
df.loc[joined_1.index.intersection(joined_2.index)]

,EntrezGene_1,GeneSymbol_1,UniProt_1,EntrezGene_2,GeneSymbol_2,UniProt_2
index,,,,,,
598,NaN,NaN,B2R8Y4,NaN,NaN,G2XKQ0
68812,NaN,NaN,Q59GP6,NaN,NaN,G2XKQ0


In [25]:
nan_before = df.isnull().sum().GeneSymbol_1 + df.isnull().sum().GeneSymbol_2
nan_after = joined_1.EntrezGene.isnull().sum() + joined_2.EntrezGene.isnull().sum() 
print("NaN Entrez Gene before mapping with UniProtKB:", nan_before)
print("NaN Entrez Gene after mapping with UniProtKB:", nan_after)
print("newly found: {} ({}%)"
      .format(nan_before-nan_after, 100-round(nan_after/nan_before, 1)*100))

NaN Entrez Gene before mapping with UniProtKB: 261
NaN Entrez Gene after mapping with UniProtKB: 175
newly found: 86 (30.0%)


In [46]:
df_entrez = df[df['EntrezGene_1'].notnull()]
df_entrez = df_entrez[df_entrez['EntrezGene_2'].notnull()]
df_entrez.EntrezGene_1 = df_entrez.EntrezGene_1.astype(int)
df_entrez.EntrezGene_2 = df_entrez.EntrezGene_2.astype(int)

print("removed PPI number: {} ({}%)".format(df.shape[0]-df_entrez.shape[0],
                                          round(100-df_entrez.shape[0]/df.shape[0]*100,1)))

# unique number
df_entrez_unique_unip = df_entrez.UniProt_1.append(df_entrez.UniProt_2)
df_entrez_unique_unip = df_entrez_unique_unip.drop_duplicates()

df_entrez_unique_entrez = df_entrez.EntrezGene_1.append(df_entrez.EntrezGene_2)
df_entrez_unique_entrez = df_entrez_unique_entrez.drop_duplicates()

df_entrez_unique_geneSymb = df_entrez.GeneSymbol_1.append(df_entrez.GeneSymbol_2)
df_entrez_unique_geneSymb = df_entrez_unique_geneSymb.drop_duplicates()

print("unique UniProt:", len(df_entrez_unique_unip))
print("unique Entrez Gene:", len(df_entrez_unique_entrez))
print("unique Gene Symbol:", len(df_entrez_unique_geneSymb))
print("\nall interactions (PPI):", df_entrez.shape[0])

removed PPI number: 1067 (0.9%)
unique UniProt: 14426
unique Entrez Gene: 14080
unique Gene Symbol: 14076

all interactions (PPI): 114914


In [29]:
# save
filename = (data_folder + 'PPI_APID_{}.csv'.
            format(datetime.datetime.now().strftime("%Y_%m_%d")))
df_entrez.to_csv(filename, sep='\t', index=False)

# PPI network adjacency matrix
around 30-40 secondes to generate a new adjacency matrix

In [30]:
import glob
from scipy.io import loadmat, savemat

In [31]:
gene_id_ppi, network = load_data.load_PPI_network(data_folder, ppi_data="APID")

 ==== load_PPI_APID
***** PPI_APID file already exists *****


In [43]:
network

<14080x14080 sparse matrix of type '<class 'numpy.float32'>'
	with 227160 stored elements in Compressed Sparse Column format>